In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('env') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/28 12:37:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
!ls

01_pyspark.ipynb       data			       lib
02_pyspark.ipynb       download_data.sh		       notes.md
03_pyspark.ipynb       fhvhv			       taxi+_zone_lookup.csv
06_spark_sql.ipynb     fhvhv_tripdata_2021-01.csv      test.txt
06_spark_sql.py        fhvhv_tripdata_2021-01.csv.gz   tmp
07_groupby_join.ipynb  fhvhv_tripdata_2021-01.parquet  zones
09_spark_sql.ipynb     fhvhv_tripdata_2021-06.csv.gz
assignment.ipynb       head.csv


In [5]:
# !wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz

In [6]:
import pandas as pd

data = pd.read_csv('fhvhv_tripdata_2021-06.csv.gz')

print(data.dtypes)
print(data.head())
print(data.shape)

dispatching_base_num      object
pickup_datetime           object
dropoff_datetime          object
PULocationID               int64
DOLocationID               int64
SR_Flag                   object
Affiliated_base_number    object
dtype: object
  dispatching_base_num      pickup_datetime     dropoff_datetime  \
0               B02764  2021-06-01 00:02:41  2021-06-01 00:07:46   
1               B02764  2021-06-01 00:16:16  2021-06-01 00:21:14   
2               B02764  2021-06-01 00:27:01  2021-06-01 00:42:11   
3               B02764  2021-06-01 00:46:08  2021-06-01 00:53:45   
4               B02510  2021-06-01 00:45:42  2021-06-01 01:03:33   

   PULocationID  DOLocationID SR_Flag Affiliated_base_number  
0           174            18       N                 B02764  
1            32           254       N                 B02764  
2           240           127       N                 B02764  
3           127           235       N                 B02764  
4           144           146  

In [7]:
df = spark.read \
        .option("header", "true") \
        .csv('fhvhv_tripdata_2021-06.csv.gz')

In [13]:
df.schema

StructType(List(StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,TimestampType,true),StructField(dropoff_datetime,TimestampType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true),StructField(Affiliated_base_number,StringType,true)))

In [4]:
from pyspark.sql import types

In [5]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.StringType(), True),
    types.StructField('DOLocationID', types.StringType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [6]:
df = spark.read \
        .option("header", "true") \
        .schema(schema) \
        .csv('fhvhv_tripdata_2021-06.csv.gz')

In [7]:
df.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                B02764|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                B02764|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                B02764|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|      N|                  null|
+--------------------+------------------

In [8]:
df = df.repartition(12)

In [9]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02875|2021-06-20 03:36:37|2021-06-20 03:47:23|         242|         213|      N|                B02875|
|              B02876|2021-06-25 08:23:10|2021-06-25 08:39:11|          77|         225|      N|                B02876|
|              B02510|2021-06-13 19:06:04|2021-06-13 19:31:06|         163|          18|      N|                  null|
|              B02872|2021-06-23 12:26:10|2021-06-23 12:52:30|         231|         181|      N|                B02872|
|              B02870|2021-06-26 12:23:44|2021-06-26 12:34:59|         173|          70|      N|                B02870|
|              B02764|2021-06-06 19:00:0

In [10]:
# df.write.parquet('data/pq/fhvhv/fhvhv_tripdata_2021-06', mode="overwrite")

In [14]:
!ls -lh data/pq/fhvhv/fhvhv_tripdata_2021-06/

total 284M
-rw-r--r-- 1 gbotemi gbotemi   0 Feb 28 13:03 _SUCCESS
-rw-r--r-- 1 gbotemi gbotemi 24M Feb 28 13:02 part-00000-72043231-97ea-470d-9606-445226944594-c000.snappy.parquet
-rw-r--r-- 1 gbotemi gbotemi 24M Feb 28 13:02 part-00001-72043231-97ea-470d-9606-445226944594-c000.snappy.parquet
-rw-r--r-- 1 gbotemi gbotemi 24M Feb 28 13:02 part-00002-72043231-97ea-470d-9606-445226944594-c000.snappy.parquet
-rw-r--r-- 1 gbotemi gbotemi 24M Feb 28 13:02 part-00003-72043231-97ea-470d-9606-445226944594-c000.snappy.parquet
-rw-r--r-- 1 gbotemi gbotemi 24M Feb 28 13:03 part-00004-72043231-97ea-470d-9606-445226944594-c000.snappy.parquet
-rw-r--r-- 1 gbotemi gbotemi 24M Feb 28 13:03 part-00005-72043231-97ea-470d-9606-445226944594-c000.snappy.parquet
-rw-r--r-- 1 gbotemi gbotemi 24M Feb 28 13:03 part-00006-72043231-97ea-470d-9606-445226944594-c000.snappy.parquet
-rw-r--r-- 1 gbotemi gbotemi 24M Feb 28 13:03 part-00007-72043231-97ea-470d-9606-445226944594-c000.snappy.parquet
-rw-r--r-- 1 gbotemi g

In [ ]:
#read parquet file and perform operation on it

In [15]:
df = spark.read.parquet('data/pq/fhvhv/fhvhv_tripdata_2021-06/')

In [16]:
df.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02884|2021-06-17 19:06:11|2021-06-17 19:36:02|          42|         169|      N|                B02884|
|              B02872|2021-06-07 09:23:51|2021-06-07 09:31:47|         242|         213|      N|                B02872|
|              B02875|2021-06-29 04:40:25|2021-06-29 05:10:30|         249|         132|      N|                B02875|
|              B02872|2021-06-07 17:15:23|2021-06-07 18:03:40|          48|         138|      N|                B02872|
|              B02510|2021-06-19 17:33:55|2021-06-19 17:44:35|         255|         112|      N|                  null|
+--------------------+------------------

In [82]:
df.registerTempTable('trips_data')
zones.registerTempTable('zone')

In [18]:
spark.sql("""
SELECT
    count(*)
FROM
    trips_data
WHERE 
    pickup_datetime >= "2021-06-15 00:00:00"
    AND
    pickup_datetime < "2021-06-16 00:00:00"
"""
).show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



In [196]:
spark.sql("""
SELECT
    dispatching_base_num,
    pickup_datetime,
    dropoff_datetime,
    CAST(CASTr(dropoff_datetime - pickup_datetime) as datetime)) AS duration,
FROM
    trips_data
LIMIT 5
"""
).show()

ParseException: 
mismatched input ')' expecting {<EOF>, ';'}(line 6, pos 58)

== SQL ==

SELECT
    dispatching_base_num,
    pickup_datetime,
    dropoff_datetime,
    CAST((dropoff_datetime - pickup_datetime) as datetime)) AS duration,
----------------------------------------------------------^^^
FROM
    trips_data
LIMIT 5


In [65]:
zones = spark.read.option("header", "true").csv('taxi+_zone_lookup.csv')

In [136]:
zones.filter(zones.LocationID > 61).show()

+----------+---------+--------------------+------------+
|LocationID|  Borough|                Zone|service_zone|
+----------+---------+--------------------+------------+
|        62| Brooklyn| Crown Heights South|   Boro Zone|
|        63| Brooklyn|       Cypress Hills|   Boro Zone|
|        64|   Queens|          Douglaston|   Boro Zone|
|        65| Brooklyn|Downtown Brooklyn...|   Boro Zone|
|        66| Brooklyn|  DUMBO/Vinegar Hill|   Boro Zone|
|        67| Brooklyn|       Dyker Heights|   Boro Zone|
|        68|Manhattan|        East Chelsea| Yellow Zone|
|        69|    Bronx|East Concourse/Co...|   Boro Zone|
|        70|   Queens|       East Elmhurst|   Boro Zone|
|        71| Brooklyn|East Flatbush/Far...|   Boro Zone|
|        72| Brooklyn|East Flatbush/Rem...|   Boro Zone|
|        73|   Queens|       East Flushing|   Boro Zone|
|        74|Manhattan|   East Harlem North|   Boro Zone|
|        75|Manhattan|   East Harlem South|   Boro Zone|
|        76| Brooklyn|       Ea

In [69]:
df_temp = df.join(zones, df.PULocationID == zones.LocationID)

In [75]:
# spark.sql("""
# SELECT count(*),
# FROM temp
# GROUP BY PULocationID
# """)

In [ ]:
spark.createDataFrame('taxi+_zone_lookup.csv').createTempView()

In [78]:
df_temp.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number',
 'LocationID',
 'Borough',
 'Zone',
 'service_zone']

In [81]:
spark.sql("""
SELECT
    count(*),
    PULocationID
FROM
    temp
GROUP BY
    PULocationID
LIMIT 5
"""
).show()

+--------+------------+
|count(1)|PULocationID|
+--------+------------+
|   50712|         125|
|  152493|           7|
|   21165|         124|
|   62409|          51|
|   69232|         205|
+--------+------------+



In [86]:
zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [154]:
spark.sql("""

SELECT
    (count(*)) as count,
    zone
    
FROM
    trips_data as t
LEFT JOIN zone as z
ON t.PULocationID = z.LocationID
GROUP BY zone
ORDER BY count DESC
LIMIT 5
"""
).show()

+------+-------------------+
| count|               zone|
+------+-------------------+
|231279|Crown Heights North|
|221244|       East Village|
|188867|        JFK Airport|
|187929|     Bushwick South|
|186780|      East New York|
+------+-------------------+



In [156]:
zones.createTempView("temp_zone")

In [167]:
spark.sql("""
SELECT *
FROM temp_zone
""").show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows

